In [56]:
import sys, os
import torch
import librosa
import numpy as np
import pandas as pd
from torch import Tensor
from scipy.io import wavfile
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import torchaudio
import numpy as np
import scipy.io.wavfile

In [60]:
class RawNetDataset(Dataset):
    def __init__(self, transform=None, mode="train",files_dir=None, base_dir="",csv_file_dir="",nb_time=59049):
        self.base_dir = base_dir
        self.mode = mode
        self.transform = transform
        self.csv_file_dir = csv_file_dir
        self.files_dir = files_dir
        csv_file = pd.read_csv(csv_file_dir,sep="\t")
        self.nb_time = nb_time
        self.classes = np.array([i for i, cls_name in enumerate(csv_file["VoxCeleb1 ID"].unique())])

    def __len__(self):
        return self.classes.shape[0] 
    
    def __getitem__(self, idx):
        filename = self.files_dir[idx]
        X, sample_rate = torchaudio.load(self.base_dir+filename)
        #print(" shape(X): ",type(X))
        label = self.classes[idx]
        self._pre_emphasis(X)
        nb_time = X.shape[1]
        if nb_time > self.nb_time:
            start_idx = np.random.randint(low = 0,
                high = nb_time - self.nb_time)
            X = X[:, start_idx:start_idx+self.nb_time]
            #print("nb_time: ",nb_time )
            #print("self.nb_time: ",self.nb_time)
        elif nb_time < self.nb_time:
            nb_dup = int(self.nb_time / nb_time) + 1
            X = np.tile(X, (1, nb_dup))[:, :self.nb_time]
            #print("taille inférieure")
        else:
            X = X
            #print("taille égale")
        print(" type(X): ",X.size())

        print('------------------------------------------------')

        return X, label

    def _pre_emphasis(self, x):
        '''
        Pre-emphasis for single channel input
        '''
        return np.asarray(x[:,1:] - 0.97 * x[:, :-1], dtype=np.float32) 



In [61]:
base_dir = '../data/wav'
list_IDs = ['/id10009/7hpSiT9_gCE/00001.wav',
        '/id10009/aFttHpeaXaQ/00001.wav',
        '/id10009/AtavJVP4bCk/00001.wav',
        '/id10009/AtavJVP4bCk/00002.wav',
        '/id10009/AtavJVP4bCk/00003.wav',
        '/id10009/AtavJVP4bCk/00004.wav',
        '/id10009/AtavJVP4bCk/00005.wav',
        '/id10009/AtavJVP4bCk/00006.wav',
        '/id10009/AtavJVP4bCk/00007.wav',
        '/id10009/AtavJVP4bCk/00008.wav',
        '/id10009/AtavJVP4bCk/00009.wav',
        '/id10009/AtavJVP4bCk/00010.wav',
        ]
csv_file_dir = '../data/target/vox1_meta.csv'

In [62]:
dataset = RawNetDataset(files_dir=list_IDs,base_dir=base_dir,csv_file_dir=csv_file_dir)
data_loader = torch.utils.data.DataLoader(dataset,
                                          batch_size=1,
                                          shuffle=True)
print(len(dataset))

1251


In [45]:
for i in range(10):
    dataset.__getitem__(i)

 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------
 type(X):  torch.Size([1, 59049])
------------------------------------------------


In [33]:
t=dataset.classes.shape[0]
print(t)

(1251,)


In [8]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets

In [9]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 

        # Non-linearity
        self.sigmoid = nn.Sigmoid()

        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)

        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)

        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [15]:
# 1. Instantiation du model
input_dim = 59049
hidden_dim = 100
output_dim = 1
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [17]:
# Instantiation de la loss function
criterion = nn.CrossEntropyLoss()

In [20]:
# optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 
print(model.parameters())
print(len(list(model.parameters())))


<generator object Module.parameters at 0x1c2e37b6d8>
4


In [ ]:
# 2. Train model
def train(num_epochs):
    iter = 0
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            # Load images with gradient accumulation capabilities
            images = images.view(-1, 28*28).requires_grad_()

            # Clear gradients w.r.t. parameters
            optimizer.zero_grad()

            # Forward pass to get output/logits
            outputs = model(images)

            # Calculate Loss: softmax --> cross entropy loss
            loss = criterion(outputs, labels)

            # Getting gradients w.r.t. parameters
            loss.backward()

            # Updating parameters
            optimizer.step()

            iter += 1

            if iter % 500 == 0:
                # Calculate Accuracy         
                correct = 0
                total = 0
                # Iterate through test dataset
                for images, labels in test_loader:
                    # Load images with gradient accumulation capabilities
                    images = images.view(-1, 28*28).requires_grad_()

                    # Forward pass only to get logits/output
                    outputs = model(images)

                    # Get predictions from the maximum value
                    _, predicted = torch.max(outputs.data, 1)

                    # Total number of labels
                    total += labels.size(0)

                    # Total correct predictions
                    correct += (predicted == labels).sum()

                accuracy = 100 * correct / total

                # Print Loss
                print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))